# This notebook solves the Huggett Model (1993) using Dolo.

This is a heterogeneous-agent model where each period agents are hit with idiosyncratic income shocks $y_t$ that follow an $AR1$ process. There are incomplete markets and agents only have access to a risk-free asset $s_t$ that pays $(1+r)s_t$ next period, where $r$ is the interest rate.

The value function for an agent with current assets $s$ and current income $y$ is: $v(y,s)=\max_{c,s'} u(c)+\beta \mathbf{E}v(y',s')$ where the expectation is taken over the value of the income shock.

The agent's budget constraint is: $c+s'=(1+r)s+y$ where s' is his asset choice next period. The agent will also be subject to a borrowing constraint: $s'\geq \bar{s}$.

Here, we define the control in the model as $a=s'-s$, i.e. $a$ is the change in assets.


In [1]:
# First import the packages
Pkg.dir("Dolo")
import Dolo
using AxisArrays
using PyPlot

. To use SymEngine run the following code: `Pkg.add("SymEngine")`

In [2]:
# get the model file
filename=("huggett_1993.yaml")

(Dolo.DiscreteMarkovProcess) in module Dolo at C:\Users\Angela\AppData\Local\JuliaPro-0.5.2.2\pkgs-0.5.2.2\v0.5\Dolo\src\numeric/processes.jl:37 overwritten at C:\Users\Angela\AppData\Local\JuliaPro-0.5.2.2\pkgs-0.5.2.2\v0.5\Dolo\src\numeric/processes.jl:157.


"huggett_1993.yaml"

In [3]:
# Convert the file into Dolo model
model=Dolo.yaml_import(filename)

name,Huggett 1993
filename,huggett_1993.yaml
Type,Equation
value,\[V_{t} = \frac{\left(c_{t}\right)^{\left(1-\sigma\right)}}{\left(1-\sigma\right)}+\beta V_{t+1}\]
expectation,\[m_{t} = \frac{\beta}{\left(c_{t+1}\right)^{\sigma}} 1+r\]
felicity,\[u_{t} = \frac{\left(c_{t}\right)^{\left(1-\sigma\right)}}{\left(1-\sigma\right)}\]
transition,\[s_{t} = a_{t-1}+s_{t-1}\]
arbitrage,\[\left(1-\beta \left(\frac{c_{t}}{c_{t+1}}\right)^{\sigma} 1+r\right)\]


Create a function to loop over guesses of the interest rate until the asset market clears.

In [4]:
function solve_r(r_val; mc_state=5, N=1000, T=1000)

    dr_guess=Dolo.ConstantDecisionRule(model.calibration[:controls])
    mc_ar = Dolo.discretize(Dolo.MarkovChain, model.exogenous; N=mc_state)
    exog_proc=Dolo.simulate(mc_ar, N,T, mc_state)
    
    return solve_r(r_val, dr_guess, exog_proc )
end   

function solve_r(r_val, dr_guess; N=1000, T=1000)
    
    mc_ar = Dolo.discretize(Dolo.MarkovChain, model.exogenous; N=mc_state)
    exog_proc=Dolo.simulate(mc_ar, N,T, mc_state)
    
    return solve_r(r_val, dr_guess, exog_proc)
    
    
end


Model


solve_r (generic function with 2 methods)

In [5]:
function solve_r(r_val, dr_guess, Index_mc, mc_ar; N=1000, T=1000)
    Dolo.set_calibration!(model, r=r_val);    
    # pass the markov chain to time_iteration
    sol=Dolo.time_iteration(model, mc_ar,dr_guess; verbose=false)
    dr=sol.dr
    conv_bool=sol.x_converged
    println(conv_bool)
    sim_armc = Dolo.simulate(model,dr, Index_mc, mc_ar)
    assets_end=zeros(N)
    for ii=1:N # number of simulations
      c=exp(sim_armc[Axis{:N}(ii), Axis{:V}(:lny)])[T]+sim_armc[Axis{:N}(ii), Axis{:V}(:s)][T]*r_val-sim_armc[Axis{:N}(ii), Axis{:V}(:a)][T]
      #assets_end[ii]=exp(sim_armc[Axis{:N}(ii), Axis{:V}(:lny)])[T]-c[end]
      assets_end[ii]=sim_armc[Axis{:N}(ii), Axis{:V}(:a)][T]
    end
    
    model_resid=mean(assets_end)
    return model_resid, dr
end

solve_r (generic function with 3 methods)

In [6]:
function asset_clear(model)
    r_grid=linspace(-0.01, 0.05,150)
    resid_vec=zeros(length(r_grid))
    it=1
    tol=0.0001
    dr_guess=Dolo.ConstantDecisionRule(model.calibration[:controls])
    max_it=150
    mc_ar = Dolo.discretize(Dolo.MarkovChain, model.exogenous; N=5)
    Index_mc=Dolo.simulate(mc_ar, 1000, 1000, 5)
        while it<max_it
            r_val=r_grid[it]
            model_resid, dr=solve_r(r_val, dr_guess, Index_mc, mc_ar) 
            resid_vec[it]=model_resid
            dr_guess=dr

            if abs(model_resid)<tol
                println(model_resid)
                println("Market Clearing with r ", r_val)
                #break
            else
                println("With r ", r_val, " the level of assets is ", model_resid)
                
            end
            it=it+1
        end
    return resid_vec
end



asset_clear (generic function with 1 method)

In [7]:
resid_vec=asset_clear(model)

true
With r -0.01 the level of assets is 0.0045799336799779026
true
With r -0.009597315436241611 the level of assets is 0.004613417333198067
true
With r -0.009194630872483222 the level of assets is 0.004646557111390716
true
With r -0.008791946308724832 the level of assets is 0.004679329969539967
true
With r -0.008389261744966443 the level of assets is 0.004711716481223484
true
With r -0.007986577181208054 the level of assets is 0.0047436955735517886
true
With r -0.007583892617449664 the level of assets is 0.0047752493957475325
true
With r -0.007181208053691275 the level of assets is 0.004806358584633653
true
With r -0.006778523489932886 the level of assets is 0.004837005413180966
true
With r -0.0063758389261744965 the level of assets is 0.004867172684038673
true
With r -0.005973154362416107 the level of assets is 0.004896841664255055
true
With r -0.005570469798657718 the level of assets is 0.004925992324224497
true
With r -0.0051677852348993285 the level of assets is 0.0049546041214007

true
With r 0.03308724832214765 the level of assets is 0.003250473978665277
true
With r 0.03348993288590604 the level of assets is 0.0031351477059323764
true
With r 0.03389261744966443 the level of assets is 0.0030205824640782445
true
With r 0.03429530201342282 the level of assets is 0.0029073020086258354
true
With r 0.034697986577181206 the level of assets is 0.0027955797788330452
true
With r 0.0351006711409396 the level of assets is 0.0026854404433252956
true
With r 0.03550335570469799 the level of assets is 0.0025763400876436275
true
With r 0.035906040268456375 the level of assets is 0.0024674432897574565
true
With r 0.03630872483221476 the level of assets is 0.002358301959934085
true
With r 0.03671140939597316 the level of assets is 0.002248984669292861
true
With r 0.037114093959731545 the level of assets is 0.002139346965741375
true
With r 0.03751677852348993 the level of assets is 0.0020288966110407407
true
With r 0.03791946308724832 the level of assets is 0.001917098593343404
tr

150-element Array{Float64,1}:
  0.00457993 
  0.00461342 
  0.00464656 
  0.00467933 
  0.00471172 
  0.0047437  
  0.00477525 
  0.00480636 
  0.00483701 
  0.00486717 
  0.00489684 
  0.00492599 
  0.0049546  
  ⋮          
 -0.00102114 
 -0.001169   
 -0.00130483 
 -0.00142471 
 -0.00150272 
 -0.00151678 
 -0.00146662 
 -0.00133718 
 -0.0011111  
 -0.000787775
 -0.000348837
  0.0        

In [ ]:
import PyPlot
plt=PyPlot
T=149
 r_grid=linspace(-0.01, 0.05,150)
plt.plot(r_grid[1:T], r_grid[1:T]*0, linestyle="--")
plt.plot(r_grid[1:T], resid_vec[1:T],  color="blue")
plt.ylim(-0.1,0.1)
plt.xlabel("Interest rate")
plt.ylabel("Asset Supply")

In [ ]:
@time sol=Dolo.time_iteration(model,verbose=true, maxit=1000)
dr=sol.dr
@time res = Dolo.time_iteration(model, dr; maxit=200)

Dolo tabulate gives us the decision rules.

In [ ]:
drtab = Dolo.tabulate(model, dr, :s) 

Now we plot the consumption policy function. We see that it is concave because of the precautionary savings motive noting as well that there is more curvature closer to the borrowing constraint.

In [ ]:
# Plot the consumption policy function
import PyPlot
plt=PyPlot
r=model.calibration.flat[:r]
print(r)
c=exp(drtab[Axis{:V}(:lny)])+drtab[:s]*r-drtab[Axis{:V}(:a)]

plt.plot(drtab[Axis{:V}(:s)],c, color="blue")
plt.xlabel("Savings")
plt.ylabel("Consumption")
plt.title("Consumption Policy Function")

### Simulations

Here we run simulations for N agents and look at the asset distribution.

In [ ]:
# Simulations
import PyPlot
plt=PyPlot


mc_ar=model.exogenous

sim_armc = Dolo.simulate(model,dr;N=1000,T=200)


T=200

In [ ]:
N=1000
n=200 # number of periods to plot
hor=linspace(1,n,n)
function plot_simulations(N::Int64,T::Int64,n::Int64,sim_armc)
    assets_end=zeros(N)
    for ii=1:N # number of simulations
          
        c=exp(sim_armc[Axis{:N}(ii), Axis{:V}(:lny)])[T-n+1:T]+sim_armc[Axis{:N}(ii), Axis{:V}(:s)][T-n+1:T]*r-sim_armc[Axis{:N}(ii), Axis{:V}(:a)][T-n+1:T]
        assets_end[ii]=exp(sim_armc[Axis{:N}(ii), Axis{:V}(:lny)])[T]-c[end]

        #assets_end[ii]=exp(sim_armc[Axis{:N}(ii), Axis{:V}(:lny)])[T]-c[end]
    
    end

    return assets_end
end




In [ ]:
assets_end=plot_simulations(N,T,n,sim_armc);

We see that the average asset holdings is very close to zero so we have market clearing.

In [ ]:
using Plots
histogram(assets_end,nbins=80,xlims=(-2.0,4.0), ylims=(0.0,120.0), label="", xlabel="Assets", ylabel="Frequency", title="Assets Distribution")


In [ ]:
mean(assets_end)